## Natural Language to SQL using Google's Gemini Pro | Python | Google AI Studio

[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

Click on the link below to open a Colab version of the notebook. You will be able to create your own version.

<a href="https://colab.research.google.com/github/bhattbhavesh91/n2sql-google-gemini/blob/main/n2sql-google-gemini-notebook.ipynb" target="_blank"><img height="40" alt="Run your own notebook in Colab" src = "https://colab.research.google.com/assets/colab-badge.svg"></a>

# Installation

In [ ]:
!pip install -q google-generativeai==0.3.1

# Imports

In [ ]:
import google.generativeai as genai
from pathlib import Path
import sqlite3

# Version

In [ ]:
genai.__version__

'0.3.1'

# Secret Key

In [ ]:
from google.colab import userdata

genai.configure(api_key = userdata.get('GEMINI_KEY'))

# Configurations

In [ ]:
# Set up the model
generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

# Model Instance

In [ ]:
model = genai.GenerativeModel(model_name = "gemini-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

# SQL Query Executor

In [ ]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()

In [ ]:
read_sql_query('SELECT * FROM fashion_products LIMIT 10;',
               "fashion_db.sqlite")

(19, 1, 'Dress', 'Adidas', "Men's Fashion", 40, 1.0431592108361825, 'Black', 'XL')
(97, 2, 'Shoes', 'H&M', "Women's Fashion", 82, 4.026416271141911, 'Black', 'L')
(25, 3, 'Dress', 'Adidas', "Women's Fashion", 44, 3.337937559377053, 'Yellow', 'XL')
(57, 4, 'Shoes', 'Zara', "Men's Fashion", 23, 1.0495229563128543, 'White', 'S')
(79, 5, 'T-shirt', 'Adidas', "Men's Fashion", 79, 4.302773408398684, 'Black', 'M')
(98, 6, 'Dress', 'Adidas', "Men's Fashion", 47, 1.3795657395330458, 'Yellow', 'L')
(16, 7, 'Jeans', 'Gucci', "Men's Fashion", 37, 1.3567503746842564, 'White', 'XL')
(63, 8, 'Sweater', 'Zara', "Kids' Fashion", 64, 4.36030328941572, 'Blue', 'XL')
(96, 9, 'Sweater', 'H&M', "Men's Fashion", 53, 4.466181876278437, 'Green', 'XL')
(36, 10, 'T-shirt', 'Zara', "Kids' Fashion", 55, 4.093234402033421, 'White', 'XL')


# Define Prompt

In [ ]:
prompt_parts_1 = [
  "You are an expert in converting English questions to SQL code! The SQL database has the name fashion_products and has the following columns - user_id, product_id, product_name, brand, category, price, color, and size.\n\nFor example,\nExample 1 - How many entries of Adidas are present?, the SQL command will be something like this\n``` SELECT COUNT(*) FROM fashion_products WHERE brand = 'Adidas';\n```\n\nExample 2 - How many XL products of Nike are there that have a rating of more than 4?\n```\nSELECT COUNT(*) FROM fashion_products WHERE brand = 'Nike' AND size = 'XL' AND \"Rating\" > 4;\n```\n\nExample 3 - \n```\nSELECT product_name FROM fashion_products WHERE price = (SELECT MAX(price) FROM fashion_products);\n```\n\nDont include ``` and \\n in the output",
]

In [ ]:
question = "Tell me the id of the most expensive T-shirt?"

In [ ]:
prompt_parts = [prompt_parts_1[0], question]
response = model.generate_content(prompt_parts)
response.text

"SELECT product_id FROM fashion_products WHERE product_name = 'T-shirt' AND price = (SELECT MAX(price) FROM fashion_products WHERE product_name = 'T-shirt');"

In [ ]:
read_sql_query("""SELECT product_id FROM fashion_products WHERE product_name = 'T-shirt' AND price = (SELECT MAX(price) FROM fashion_products WHERE product_name = 'T-shirt');
""",
               "fashion_db.sqlite")

(938,)


# Combine it into Function

In [ ]:
def generate_gemini_response(question, input_prompt):
    prompt_parts = [input_prompt, question]
    response = model.generate_content(prompt_parts)
    output = read_sql_query(response.text, "fashion_db.sqlite")
    return output

In [ ]:
generate_gemini_response("How many products of Nike are there?",
                         prompt_parts_1[0])

(214,)
